<a href="https://colab.research.google.com/github/BobGanti/ColabNotebooks/blob/main/pdf_to_text_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unstructured To Structured Dataset Creation With Few-Shot Learning Of Gen Models

In [ ]:
!pip install -q imbalanced-learn \
PyMuPDF \
openai \
PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
import os
import re
import nltk
import openai
import PyPDF2
import pymupdf
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive, userdata
drive = drive.mount('/content/drive', force_remount=True)

DATA_DIR = userdata.get('ROOT_DIR')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
arxiv_DIR = DATA_DIR + "/arxiv"
PDF_DIR = arxiv_DIR + "/PDFs"

openai.api_key = OPENAI_API_KEY

Mounted at /content/drive


### Questions Generation

In [ ]:
import json
import csv

openai.api_key = OPENAI_API_KEY

def generate_question(text_chunk):

    prompt = f"""
    You are an expert in analysing text, determining the context and topic being discussed in the text and generating relevant question.
    Learn from the given examples 1, 2, 3. Then analyse the given text chunk and generate the most appropriate question with 100% cosine similarity and BM25r can be extracted from the text chunk.

    Example 1:
    Text: 'Large language models (LLMs) inevitably exhibit hallucinations since the accuracy of generated texts cannot be secured solely by the parametric knowledge they encapsulate. Although retrieval-augmented generation (RAG) is a practicable complement to LLMs, it relies heavily on the relevance of retrieved documents, raising concerns about how the model behaves if retrieval goes wrong.'
    Question: What issue do large language models (LLMs) inevitably exhibit, and what is a concern related to retrieval-augmented generation (RAG)?
    Similarity Score: 0.9327

    Example 2:
    Text: 'To this end, we propose the Corrective Retrieval Augmented Generation (CRAG) to improve the robustness of generation. Specifically, a lightweight retrieval evaluator is designed to assess the overall quality of retrieved documents for a query, returning a confidence degree based on which different knowledge retrieval actions can be triggered.'
    Question: What is the purpose of the Corrective Retrieval Augmented Generation (CRAG) and what specific component is designed to assess the quality of retrieved documents?
    Similarity Score: 0.9864

    Example 3:
    Text: 'Since retrieval from static and limited corpora can only return sub-optimal documents, large-scale web searches are utilized as an extension for augmenting the retrieval results. Besides, a decompose-then-recompose algorithm is designed for retrieved documents to selectively focus on key information and filter out irrelevant information in them.'
    Question: What method is proposed to augment the retrieval results from static and limited corpora, and what algorithm is designed to process the retrieved documents?
    similarity Score: 0.9379

    Now, read the following text chunk and generate the most appropriate question:

    Text: '{text_chunk}'
    Question:
    """
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert in question generator."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        temperature=0.5,
        n=1,
        stop=None
    )
    question = str(response.choices[0].message.content).strip()
    return question

def generate_text(question, label):
    if label == 'ambiguous':
        prompt = f"""
        Generate a text chunk that somewhat relates to the following question but can only provide context that captures only less than 50% down to 15% of the potential answer.

        Example 1:
        Question: What issue do large language models (LLMs) inevitably exhibit, and what is a concern related to retrieval-augmented generation (RAG)?
        Text: Advances in large language models often pivot around optimizing computational efficiency and reducing energy consumption. These models are increasingly deployed in real-time applications, where their ability to adapt to streaming data becomes paramount, diverging from concerns typically associated with static data sets.
        Cosine Similarity Score: 38.82%

        Example 2:
        Question: What is the purpose of the Corrective Retrieval Augmented Generation (CRAG) and what specific component is designed to assess the quality of retrieved documents?
        Text: The exploration of new algorithms in text generation frequently targets the enhancement of linguistic diversity and the reduction of biases. Key components often involve neural network architectures that focus on generative adversarial networks, which work to create more nuanced and varied text outputs.
        Cosine Similarity Score: 33.18%

        Example 3:
        Question: What method is proposed to augment the retrieval results from static and limited corpora, and what algorithm is designed to process the retrieved documents?
        Text: Current trends in database management focus on improving security protocols and data encryption methods to safeguard information against cyber threats. Enhanced security measures include the implementation of advanced encryption standards and real-time intrusion detection systems.
        Cosine Similarity Score: 24.83%

        Example 4:
        Question: How is CRAG described in terms of integration with RAG-based approaches, and what do experiments show about its performance?
        Text: Discussions around CRAG often shift towards its potential applications in non-text-based fields, such as super market stuff recognition and video analysis. These applications explore how cross-modal data integration can lead to more robust models, with experiments often delving into the use of deep learning for feature extraction.
        Cosine Similarity Score: 50%

        Now, read the following question and generate a somewhat related text chunk:

        Question: "{question}"
        Text:
        """
    elif label == 'irrelevant':
        prompt = f"""
        Generate a text chunk that is not related to the following question and cannot be answered based on the text.

        Example 1:
        Question: What issue do large language models (LLMs) inevitably exhibit, and what is a concern related to retrieval-augmented generation (RAG)?
        Text: Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods with the help of chlorophyll. This process is crucial for converting solar energy into chemical energy.

        Example 2:
        Question: What is the purpose of the Corrective Retrieval Augmented Generation (CRAG) and what specific component is designed to assess the quality of retrieved documents?
        Text: The Roman Empire's history is marked by significant events and figures that shaped Western civilization. From the rise of Julius Caesar to the fall of the empire, each period offers insights into the complexities of ancient governance.

        Example 3:
        Question: What method is proposed to augment the retrieval results from static and limited corpora, and what algorithm is designed to process the retrieved documents?
        Text: Quantum mechanics explores the behavior of particles at the atomic and subatomic levels. It reveals the fundamental principles governing the interactions and properties of matter and energy, challenging classical physics notions.

        Example 4:
        Question: How is CRAG described in terms of integration with RAG-based approaches, and what do experiments show about its performance?
        Text: Marine biology studies the ecosystems and organisms within the ocean. Researchers focus on understanding marine life forms, their interactions, and the impact of human activities on marine environments.

        Now, read the following question and generate an unrelated text chunk:

        Question: "{question}"
        Text:
        """
    else:
        raise ValueError("Invalid label. Must be 'ambiguous' or 'irrelevant'.")

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert in text generation."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.5,
        n=1,
        stop=None
    )

    return str(response.choices[0].message.content).strip()

def calculate_relevance_score_gpt(question, text):
    prompt = f"""
    Evaluate the relevance of the following text in answering the given question. Provide a score (value only with no text) between 0 and 100, where 100 means completely relevant and 0 means completely irrelevant.

    Question: {question}
    Text: {text}
    Score:
    """
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert in evaluating text relevance."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=10,
        temperature=0.5,
        n=1,
        stop=["Score:"]
    )

    score = str(response.choices[0].message.content).strip()
    return score

# test
question = "What issue do large language models (LLMs) face regarding the accuracy of generated texts, and what concern is associated with retrieval-augmented generation (RAG)?"
text_chunk = "Large language models (LLMs) inevitably exhibit hallucinations since the accuracy of generated texts cannot be secured solely by the parametric knowledge they encapsulate. Although retrieval-augmented generation (RAG) is a practicable complement to LLMs, it relies heavily on the relevance of retrieved documents, raising concerns about how the model behaves if retrieval goes wrong."
score = calculate_relevance_score_gpt(question, text_chunk)
print(f"Relevance score: {score}")


### OpenSource Transformers

### With All Mini

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embeddings(texts):
    """Get embeddings for a list of texts using a pre-trained model."""
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**inputs)
    embeddings = model_output.last_hidden_state.mean(dim=1)
    return embeddings

def calculate_relevance_score_mini(question, text_chunk):
    """Calculate the relevance score between a question and a text chunk."""
    embeddings = get_embeddings([question, text_chunk])
    question_embedding = embeddings[0].unsqueeze(0)
    text_chunk_embedding = embeddings[1].unsqueeze(0)
    score = cosine_similarity(question_embedding, text_chunk_embedding)[0][0]
    return score

# test
question = "What issue do large language models (LLMs) face regarding the accuracy of generated texts, and what concern is associated with retrieval-augmented generation (RAG)?"
text_chunk = "Large language models (LLMs) inevitably exhibit hallucinations since the accuracy of generated texts cannot be secured solely by the parametric knowledge they encapsulate. Although retrieval-augmented generation (RAG) is a practicable complement to LLMs, it relies heavily on the relevance of retrieved documents, raising concerns about how the model behaves if retrieval goes wrong."
score = calculate_relevance_score_mini(question, text_chunk)
print(f"Relevance score: {score}")


Relevance score: 95


### With BERT

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Function to calculate BERT-based similarity
def calculate_bert_similarity(question, text):
    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Encode the texts
    encoded_text = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    encoded_question = tokenizer(question, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # Get BERT embeddings
    with torch.no_grad():
        text_embeddings = model(**encoded_text).pooler_output
        question_embeddings = model(**encoded_question).pooler_output

    # Calculate cosine similarity
    similarity = cosine_similarity(text_embeddings, question_embeddings)[0][0]
    return similarity

# test
question = "What issue do large language models (LLMs) face regarding the accuracy of generated texts, and what concern is associated with retrieval-augmented generation (RAG)?"
text_chunk = "Large language models (LLMs) inevitably exhibit hallucinations since the accuracy of generated texts cannot be secured solely by the parametric knowledge they encapsulate. Although retrieval-augmented generation (RAG) is a practicable complement to LLMs, it relies heavily on the relevance of retrieved documents, raising concerns about how the model behaves if retrieval goes wrong."
score = calculate_relevance_score_gpt(question, text_chunk)
print(f"Relevance score: {score}")


### Creating The Documents From PDFs

In [ ]:
import pymupdf
import os
import re

def extract_text_from_pdf(file_path):
    # Open the PDF file
    pdf_document = pymupdf.open(file_path)
    text = ""

    # Extracting text from each page
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text += page.get_text()

    return text

def clean_text(text):
    # Defining patterns to exclude sections like References
    exclude_patterns = [
        r'\bReferences\b.*',  # Matching "References" and everything after
        r'\bBibliography\b.*',  # Matching "Bibliography" and everything after
         r'^\s*$',  # Matching empty lines
    ]
    for pattern in exclude_patterns:
        text = re.sub(pattern, '', text, flags=re.DOTALL)

    return text

def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = min(start + chunk_size, len(words))
        chunk = words[start:end]
        chunks.append(' '.join(chunk))
        start += chunk_size - overlap
    return chunks

def process_pdfs_in_directory(directory_path):
    document_texts = []
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.pdf'):
            file_path = os.path.join(directory_path, filename)
            text = extract_text_from_pdf(file_path)
            cleaned_text = clean_text(text)
            chunks = chunk_text(cleaned_text)
            document_texts.append(
                {'metadata':file_path,
                 'title':filename,
                 'chunks':chunks
                }
            )
    return document_texts


### The QC (Question-Context) Dataset

In [ ]:

directory_path = PDF_DIR
document_texts = process_pdfs_in_directory(directory_path)

qc = []
for doc_text in document_texts:
    for i, chunk in enumerate(doc_text['chunks']):
        text_filename = doc_text['title'].replace('.pdf', f'_{i+1}.txt')
        with open(os.path.join(directory_path, text_filename), 'w', encoding='utf-8') as text_file:
            text_file.write(chunk)

        doc_question = generate_question(chunk)
        qc.append({'question': doc_question, 'context': chunk})

print("Text extraction, cleaning, and chunking complete.")


for doc in document_texts:
    for chunk in doc['chunks']:
        doc_question = generate_question(chunk)
        qc.append(
            {
                'question': doc_question,
                'context': chunk
            }
        )

data = pd.DataFrame(qc)
data.to_csv(DATA_DIR+'/arxiv_finetuning_qa.csv', index=False, escapechar='\\')

print(data.shape)


Text extraction, cleaning, and chunking complete.


#### Load Locally Save QC Pairs

In [ ]:
import pandas as pd

def build_records(qa, thresholds):
    dataset = []
    label = ""
    questions = qa['question'].tolist()
    contexts = qa['context'].tolist()

    for i, q in enumerate(questions):
        rel_record = {
            "question": q,
            "context": contexts[i],
            "score": int(calculate_relevance_score_gpt(q, contexts[i])),
            "label": "relevant"
        }
        dataset.append(rel_record)

        amb_text = generate_text(q, 'ambiguous')
        amb_score = int(calculate_relevance_score_gpt(q, amb_text))
        if amb_score >= thresholds[1]:
            label = 'relevant'
        elif thresholds[0] <= amb_score < thresholds[1]:
            label = 'ambiguous'
        else: label = 'irrelevant'

        amb_record = {
            "question": q,
            "context": amb_text,
            "score": amb_score,
            "label": label
        }
        dataset.append(amb_record)

        irr_text = generate_text(q, 'irrelevant')
        irr_score = int(calculate_relevance_score_gpt(q, irr_text))
        if irr_score >= thresholds[0]:
            label = 'ambiguous'
        else:
            label = 'irrelevant'
        irr_record = {
            "question": q,
            "context": irr_text,
            "score": irr_score,
            "label": label
            }
        dataset.append(irr_record)

    return dataset

thresholds = [25, 60]
qc = pd.read_csv(DATA_DIR+'/arxiv_finetuning_qa.csv')
print(qc.shape)


data = build_records(qc, thresholds)
df = pd.DataFrame(data)

# Saving Dataset
df.to_csv(DATA_DIR+'/arxiv_finetuning_dataset.csv', index=False, escapechar='\\')


In [ ]:

df = pd.read_csv(DATA_DIR+'/arxiv_finetuning_dataset.csv')
print(df.shape)
df.drop_duplicates()

counts = df['label'].value_counts()
print(counts)

df.head()

(3594, 4)
label
irrelevant    1798
relevant      1198
ambiguous      598
Name: count, dtype: int64


,question,context,score,label
0,What two content modeling approaches are compa...,Document Embedding for Scientiﬁc Articles: Eﬃc...,95,relevant
1,What two content modeling approaches are compa...,The ongoing research in machine learning empha...,10,irrelevant
2,What two content modeling approaches are compa...,The Great Wall of China is one of the most ico...,0,irrelevant
3,What are the main advantages of using word emb...,the expense of 3.7 times more memory requireme...,85,relevant
4,What are the main advantages of using word emb...,The evolution of natural language processing h...,45,ambiguous


### Under Sampling

In [ ]:

import pandas as pd
from sklearn.utils import resample, shuffle

df = df.drop_duplicates()

print("Class distribution before balancing:")
print(df['label'].value_counts())

df_relevant = df[df['label'] == 'relevant']
df_irrelevant = df[df['label'] == 'irrelevant']
df_ambiguous = df[df['label'] == 'ambiguous']

min_size = len(df_ambiguous)

df_relevant_downsampled = resample(df_relevant, replace=False, n_samples=min_size, random_state=123)
df_irrelevant_downsampled = resample(df_irrelevant, replace=False, n_samples=min_size, random_state=123)

df_dnbalanced = pd.concat([df_relevant_downsampled, df_irrelevant_downsampled, df_ambiguous])

df_dnbalanced = shuffle(df_dnbalanced, random_state=123)


# Save Under Sampled Data
df_dnbalanced.to_csv(DATA_DIR+'/arxiv_finetuning_dataset_dnbalanced.csv', index=False, escapechar='\\')

print("Class distribution after balancing:")
print(df_dnbalanced['label'].value_counts())



In [ ]:
df_dnbalanced = pd.read_csv(DATA_DIR+'/arxiv_finetuning_dataset_dnbalanced.csv')
counts = df_dnbalanced['label'].value_counts()
print(counts)
df_dnbalanced.head()

label
relevant      598
irrelevant    598
ambiguous     598
Name: count, dtype: int64


,question,context,score,label
0,What retrieval model is used across all differ...,"the same, and Acc measures whether the predict...",100,relevant
1,What were the two choices of the pre-training ...,"et al., 2017) Sparse Retr.+DocReader N/A - 20....",100,relevant
2,What potential improvement is suggested for en...,improved by weighing them with the TFIDF value...,85,relevant
3,What modification is made to the attention lay...,The Great Wall of China is one of the most ico...,0,irrelevant
4,What findings were derived from the ablation s...,The Great Wall of China is one of the most ico...,0,irrelevant


### Over Sampling

In [ ]:
import pandas as pd
import random
from sklearn.utils import shuffle

df = df.drop_duplicates()

print("Class distribution before balancing:")
print(df['label'].value_counts())

df_relevant = df[df['label'] == 'relevant']
df_irrelevant = df[df['label'] == 'irrelevant']
df_ambiguous = df[df['label'] == 'ambiguous']

def augment_text(text):
    words = text.split()
    if len(words) > 1:
        # Randomly choose a modification method
        mod_type = random.choice(['reverse', 'swap', 'insert'])
        if mod_type == 'reverse':
            index = random.randint(0, len(words) - 1)
            words[index] = words[index][::-1]
        elif mod_type == 'swap' and len(words) > 2:
            idx1, idx2 = random.sample(range(len(words)), 2)
            words[idx1], words[idx2] = words[idx2], words[idx1]
        elif mod_type == 'insert':
            index = random.randint(0, len(words))
            words.insert(index, random.choice(words))
    return ' '.join(words)

# Number of samples to generate
n_samples_irrelevant = len(df_irrelevant)
n_samples_relevant = n_samples_irrelevant - len(df_relevant)
n_samples_ambiguous = n_samples_irrelevant - len(df_ambiguous)

new_relevant = df_relevant.sample(n=n_samples_relevant, replace=True).copy()
new_relevant['question'] = new_relevant['question'].apply(augment_text)
new_relevant['context'] = new_relevant['context'].apply(augment_text)

new_ambiguous = df_ambiguous.sample(n=n_samples_ambiguous, replace=True).copy()
new_ambiguous['question'] = new_ambiguous['question'].apply(augment_text)
new_ambiguous['context'] = new_ambiguous['context'].apply(augment_text)

df_upbalanced = pd.concat([df_irrelevant, df_relevant, df_ambiguous, new_relevant, new_ambiguous])

# Shuffle the balanced DataFrame
df_upbalanced = shuffle(df_upbalanced, random_state=123)
# Save Over Sampled Data
df_upbalanced.to_csv(DATA_DIR+'/arxiv_finetuning_dataset_upbalanced.csv', index=False, escapechar='\\')


In [ ]:
df_upbalanced = pd.read_csv(DATA_DIR+'/arxiv_finetuning_dataset_upbalanced.csv')
counts = df_upbalanced['label'].value_counts()
print(counts)
df_upbalanced.head()


label
relevant      1798
ambiguous     1798
irrelevant    1798
Name: count, dtype: int64


,question,context,score,label
0,What modifications can be made to the odd-size...,if end do end if end function Odd-size merging...,85,relevant
1,What is the main advantage of using a fused k-...,The development of advanced kernel methods has...,25,ambiguous
2,"What are the differences in memory usage, calc...","In the realm of natural language processing, v...",30,ambiguous
3,What notable misclassifications are revealed b...,"In the realm of machine learning, the evaluati...",25,ambiguous
4,What is the main question being investigated r...,or an end of questions token. 4.2 Question Ans...,85,relevant


### Prepare for HuggingFace

In [ ]:
# dataset_path = "/content/drive/MyDrive/Datasets/arxiv_finetuning_dataset_upbalanced.csv"
dataset_path = "/content/drive/MyDrive/Datasets/arxiv_finetuning_dataset_dnbalanced.csv"
df = pd.read_csv(dataset_path)
print(df.shape)

df = df.to_dict(orient='records')

with open("./train.jsonl", "w") as f:
    for line in df:
        f.write(json.dumps(line) + '\n')